In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/black-friday-sales-eda/train.csv')
df.head(10)

# Statistical Summary

In [ ]:
df.shape

Damn! that's a huge dataset

In [ ]:
df['Occupation'].unique()

The numbers here represent category of occupation and not the number of jobs a person has

In [ ]:
df.info()

In [ ]:
# drop unnecessary columns
df.drop(['User_ID','Product_ID'],inplace=True,axis=1)

In [ ]:
df['Age'].unique()

In [ ]:
df['City_Category'].unique()

In [ ]:
df['Product_Category_1'].unique()

In [ ]:
df['Product_Category_2'].unique()

In [ ]:
df['Product_Category_3'].unique()

# Data Preprocessing

In [ ]:
#Transform Gender Values
df['Gender'] = df['Gender'].map({'F':1,"M":0})
df['Gender'].head(5)

In [ ]:
df['City_Category'] = df['City_Category'].map({'A':0,"B":1,"C":2})

In [ ]:
sorted_age = sorted(df['Age'].unique())
dict = {}

def split_age(v):
    global dict
    
    if v == "55+":
        dict.update({v:"55"})
    else:
        split_v = v.split('-')
        dict.update( { v : (int(split_v[0]) + int(split_v[1])) / 2} )
    return dict

convert = list(map(lambda v : split_age(v), sorted_age))[-1]

## Check for Null Values

In [ ]:
df.isna().sum()

Looks like there are a lot of null values in Product Category 2 and 3

In [ ]:
# Handling NaN values in Product_Category_2
df['Product_Category_2'] = df.groupby(['Age','Gender','Occupation','Marital_Status'], sort=False,group_keys=False)['Product_Category_2'].apply(lambda x: x.fillna(x.mode().iloc[0]))

The above code will not work for Product_Category_3 because there are groups with all missing values that is null values, this creates a situation that leads to an empty mode calculation, therefore I will check for atleast one non-null value in the group and then perform further calculation

In [ ]:
data = df.groupby(['Age','Gender','Occupation','Marital_Status'],group_keys=False)

def impute_product_category_3(group):
    if group['Product_Category_3'].notna().any():
        group['Product_Category_3'] = group['Product_Category_3'].fillna(group['Product_Category_3'].mode().iloc[0])
    return group

df = data.apply(impute_product_category_3)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Age'] = df['Age'].replace(convert)

In [ ]:
df.head(10)

According to the above distribution plot, the purchase column does not have a normal distribution.

# Removing Outliers

In [ ]:
# Check for outliers
sns.boxplot(x="Age", y="Purchase", data=df, palette='rainbow')

The Above plot shows presence of outliers above 20000

Let's look a the total number of outliers present in this distribution

In [ ]:
def checking_outliers(column):
    # Group the data
    data = df.groupby(column)['Purchase']

    # Calculate the IQR for 'Purchase' in the Age Group
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    IQR = q3-q1

    #Calculate the upper and lower bounds
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR
    
    #Identify the outliers
    outliers_count = (df['Purchase'] < df[column].replace(lower_bound)) | (df['Purchase'] > df[column].replace(upper_bound))
    outliers_by_col = outliers_count.groupby(df[column]).sum()
    print(f'Outliers in the {column} and Purchase distribution:\n {outliers_by_col}')
    return lower_bound, upper_bound

lower_bound,upper_bound = checking_outliers('Age')

These are the outliers present in the Purchase column with respect to each age group

In [ ]:
def remove_outliers(column):
    # Group the data
    data_grouped = df.groupby(column)['Purchase']

    # Calculate the IQR for 'Purchase' in the Age Group
    q1 = data_grouped.quantile(0.25)
    q3 = data_grouped.quantile(0.75)
    IQR = q3 - q1

    # Calculate the upper and lower bounds
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR
    
    # Identify the outliers
    outliers_count = (df['Purchase'] < data_grouped.transform(lambda x: x.replace(lower_bound.loc[x.name])))
    outliers_count |= (df['Purchase'] > data_grouped.transform(lambda x: x.replace(upper_bound.loc[x.name])))
    outliers_by_col = outliers_count.groupby(df[column]).sum()
    print(f'Outliers in the {column} and Purchase distribution:\n{outliers_by_col}')
    return lower_bound, upper_bound

# Call the function to get lower and upper bounds for each age group
lower_bound, upper_bound = remove_outliers('Age')

# Handle outliers by clipping 'Purchase' values within the lower and upper bounds for each age group
df['Purchase'] = df.groupby('Age',group_keys=False)['Purchase'].apply(lambda x: np.clip(x, lower_bound.loc[x.name], upper_bound.loc[x.name]))

# Now the 'Purchase' column should have outliers handled within each age group


There are no more outliers in the data

In [ ]:
# Check for outliers
sns.boxplot(x="Age", y="Purchase", data=df, palette='rainbow')

In [ ]:
# Check for outliers
sns.boxplot(x="Gender", y="Purchase", data=df, palette='rainbow')

There are some outliers in the Female Category

In [ ]:
lower_bound,upper_bound = checking_outliers('Gender')

In [ ]:
# Call the function to get lower and upper bounds for each age group
lower_bound, upper_bound = remove_outliers('Gender')

# Handle outliers by clipping 'Purchase' values within the lower and upper bounds for each age group
df['Purchase'] = df.groupby('Gender',group_keys=False)['Purchase'].apply(lambda x: np.clip(x, lower_bound.loc[x.name], upper_bound.loc[x.name]))

In [ ]:
sns.boxplot(data=df, x="Occupation",y="Purchase")

In [ ]:
lower_bound, upper_bound = checking_outliers('Occupation')

In [ ]:
lower_bound, upper_bound = remove_outliers('Occupation')
df['Purchase'] = df.groupby('Occupation',group_keys=False)['Purchase'].apply(lambda x: np.clip(x, lower_bound.loc[x.name], upper_bound.loc[x.name]))

Outliers have been removed

# Data Visualization

In [ ]:
sns.kdeplot(df['Purchase'])
plt.title("Purchase Distribution")
plt.xlabel("Purchase")
plt.ylabel("Density")
plt.xlim(0, df['Purchase'].max())  # Set x-axis limit to include the maximum value of 'Purchase'
plt.show()

From the above distribution we can see that the maximum purchase amount is around 7500

In [ ]:
sns.barplot(data=df,x='Gender',y="Purchase",)
plt.title("Gender-Purchase Distribution Plot")
plt.show()

Male Purchase seems to be more than Female Purchase

In [ ]:
plt.figure(figsize=(10,6))
a = sns.barplot(data=df,x='Age',y='Purchase',palette = "rocket")
plt.bar_label(a.containers[0])
plt.title("Age-Purchase Distribution")
plt.show()

There not a lot of variations between the purchase rate of different age groups, but most purchases happends to be from people above 50

In [ ]:
plt.figure(figsize=(20,6))
a = sns.barplot(data=df,x='Occupation',y='Purchase',palette = "ocean")
plt.bar_label(a.containers[0])
plt.title("Occupation-Purchase Distribution")
plt.show()

Highest purchase is done by people with Occupation 17

Lowest purchase is done by people with Occupation 9

In [ ]:
plt.figure(figsize=(6,6))
a = sns.barplot(data=df,x='City_Category',y='Purchase',palette = "ocean")
plt.bar_label(a.containers[0])
plt.title("City-Purchase Distribution")
plt.show()

In [ ]:
sorted_categories = df.groupby('Product_Category_1')['Purchase'].mean().sort_values(ascending=False).index

# Bar plot with values sorted based on mean 'Purchase' in descending order
plt.figure(figsize=(20, 6))
a = sns.barplot(data=df, x='Product_Category_1', y='Purchase', palette='coolwarm', order=sorted_categories)
plt.bar_label(a.containers[0])
plt.title("Product_Category_1-Purchase Distribution (Sorted by Purchase)")
plt.show()

In Product Category 1 the 10th product has been purchase maximum time

In Product Category 1 the 19th product has been purchase least time

In [ ]:
sorted_categories = df.groupby('Product_Category_2')['Purchase'].mean().sort_values(ascending=False).index

# Bar plot with values sorted based on mean 'Purchase' in descending order
plt.figure(figsize=(20, 6))
a = sns.barplot(data=df, x='Product_Category_2', y='Purchase', palette='coolwarm', order=sorted_categories)
plt.bar_label(a.containers[0])
plt.title("Product_Category_2 - Purchase Distribution (Sorted by Purchase)")
plt.show()

In Product Category 2 the 10th product has been purchase maximum time

In Product Category 2 the 7th product has been purchase least time

In [ ]:
sorted_categories = df.groupby('Product_Category_3')['Purchase'].mean().sort_values(ascending=False).index

# Bar plot with values sorted based on mean 'Purchase' in descending order
plt.figure(figsize=(20, 6))
a = sns.barplot(data=df, x='Product_Category_3', y='Purchase', palette='coolwarm', order=sorted_categories)
plt.bar_label(a.containers[0])
plt.title("Product_Category_3 - Purchase Distribution (Sorted by Purchase)")
plt.show()

In Product Category 3 the 3rd product has been purchase maximum time

In Product Category 3 the 16th product has been purchase least time

In [ ]:
# Count the occurrences of each category in 'City_Category'
city_category_counts = df['City_Category'].value_counts()

# Create a pie chart using plt.pie()
plt.figure(figsize=(5,5))
plt.pie(city_category_counts, labels=city_category_counts.index, autopct='%1.1f%%', colors=['skyblue', 'lightgreen', 'lightcoral'])
plt.title("City Categories")
plt.axis('equal')  # Equal aspect ratio ensures that the pie is drawn as a circle.
plt.show()

most of the population belongs to the city B

### Purchase Distribution by each column

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your dataset is stored in a DataFrame called 'df'

# Columns for which you want to create subplots
columns_for_subplots = ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status',
                        'Product_Category_1', 'Product_Category_2', 'Product_Category_3']

# Create subplots with 3 rows and 3 columns (adjust the size as needed)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(25, 20))
plt.subplots_adjust(hspace=0.5)  # Adjust the horizontal space between subplots

# Loop through each column and create a subplot
for i, column in enumerate(columns_for_subplots):
    # Determine the row and column index for each subplot
    row_index = i // 3
    col_index = i % 3
    
    # Plot the distribution of 'Purchase' for each column
    sns.histplot(data=df, x='Purchase', hue=column, kde=True, ax=axes[row_index, col_index])
    axes[row_index, col_index].set_title(f'Purchase Distribution by {column}')
    axes[row_index, col_index].set_xlabel('Purchase')
    axes[row_index, col_index].set_ylabel('Count')

# Show the subplots
plt.tight_layout()
plt.show()


### Black_Friday Analytics

In [ ]:
# Average purchase of each gender population in each city
def city_gender(gender):
    data = df[df['Gender']==gender]
    city_gender = data.groupby('City_Category').agg({'Gender':'count','Purchase':'mean','Age':pd.Series.mode}).reset_index()
    
    Product_min = df.groupby('City_Category')[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].min().reset_index()
    Product_min.rename(columns = {'Product_Category_1':'Min_Product_1','Product_Category_2':'Min_Product_2','Product_Category_3':'Min_Product_3'},inplace=True)
    city_gender = city_gender.set_index('City_Category').join(Product_min.set_index('City_Category')[['Min_Product_1', 'Min_Product_2', 'Min_Product_3']]).reset_index()
    
    Product_max = df.groupby('City_Category')[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].max().reset_index()
    Product_max.rename(columns = {'Product_Category_1':'Max_Product_1','Product_Category_2':'Max_Product_2','Product_Category_3':'Max_Product_3'},inplace=True)
    city_gender = city_gender.set_index('City_Category').join(Product_max.set_index('City_Category')[['Max_Product_1', 'Max_Product_2', 'Max_Product_3']])
    return city_gender

In [ ]:
city_gender(0)

The above table shows that highest purchase rate of male population is from the city C that is 9903 approx

Age group with highest purchase rate = around 30

It also shows the maximun and minimum product category bought in each city

In [ ]:
city_gender(1)

The highest purchase of Female population is from the city C that is 9111 approx

Age group with highest purchase rate = around 30

It also shows the maximun and minimum product category bought in each city